In [1]:
import os

In [2]:
%pwd

'd:\\DataScience\\DS\\Neural Networks\\tensor_flow\\end-to-end-chestClassificationDL\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DataScience\\DS\\Neural Networks\\tensor_flow\\end-to-end-chestClassificationDL'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/ikram2500/end-to-end-chestClassification-mlflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "ikram2500"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "bc4a05ef4f1a8239f5992d3a59f226a7d0c22822"

In [6]:
import dagshub

In [7]:
import dagshub
dagshub.init(repo_owner='ikram2500', repo_name='end-to-end-chestClassification-mlflow-DVC', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as ikram2500

Initialized MLflow to track repo "ikram2500/end-to-end-chestClassification-mlflow-DVC"

Repository ikram2500/end-to-end-chestClassification-mlflow-DVC initialized!

In [8]:
import tensorflow as tf

In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str 
    params_image_size: list
    params_batch_size: int
    

In [11]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class CofigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
          eval_config = EvaluationConfig(
                path_of_model="artifacts/training/model.h5", # pyright: ignore[reportArgumentType]
                training_data="artifacts/data_ingestion/Chest-Ct-Scan-Data", # pyright: ignore[reportArgumentType]
                mlflow_uri="https://dagshub.com/ikram2500/end-to-end-chestClassification-mlflow-DVC.mlflow",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
            
          )
          return eval_config

In [13]:
import tensorflow as tf 
from pathlib import Path 
import mlflow 
import mlflow.keras 
from urllib.parse import urlparse 

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name= "VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [15]:
try:
    config = CofigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2025-08-29 19:20:38,586: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-29 19:20:38,601: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-29 19:20:38,603: INFO: common: created directory at: artifacts]
Found 80 images belonging to 2 classes.
5/5 [==============================] - 9s 2s/step - loss: 14.4154 - accuracy: 0.4500
[2025-08-29 19:20:48,468: INFO: common: json file saved at : scores.json]


2025/08/29 19:20:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-08-29 19:20:51,062: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\TECHZO~1\AppData\Local\Temp\tmp1weg6aw0\model\data\model\assets
[2025-08-29 19:20:51,683: INFO: builder_impl: Assets written to: C:\Users\TECHZO~1\AppData\Local\Temp\tmp1weg6aw0\model\data\model\assets]


c:\miniconda3\envs\tensorenv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/08/29 19:22:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
